In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [2]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tqdm import tqdm

In [3]:
import albumentations as a

In [4]:
model1=load_model('../input/plant-models/plant_models/densenet_tuning_0.h5')
model2=load_model('../input/plant-models/plant_models/densenet_tuning_1.h5')
model3=load_model('../input/plant-models/plant_models/densenet_tuning_2.h5')
model4=load_model('../input/plant-models/plant_models/densenet_tuning_3.h5')
model5=load_model('../input/plant-models/plant_models/densenet_tuning_4.h5')

In [5]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 386, 386, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 190, 190, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 190, 190, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

## without tta

In [6]:
df=pd.read_csv('../input/plant-models/plant_models/df_kfold.csv')

In [7]:
df.head()

,image_id,class,kfolds
0,images/Train_0.jpg,3,0
1,images/Train_1.jpg,1,0
2,images/Train_2.jpg,0,0
3,images/Train_3.jpg,2,0
4,images/Train_4.jpg,0,0


In [9]:
loss_fn=tf.keras.losses.CategoricalCrossentropy()
loss=[]
for i in tqdm(range(5)):
    temp=df[df['kfolds']==i].drop('kfolds',axis=1).reset_index(drop=True)
    y_true=temp['class'].values
    y_true=tf.one_hot(y_true,depth=4,dtype=tf.float32)
    predictions=np.empty((y_true.shape[0],4),dtype=float)
    for idx,name in enumerate(temp['image_id'].values):
        img=load_img('../input/plant-pathology-2020-fgvc7/'+name,target_size=(380,380))
        img=img_to_array(img)/255.0
        img=np.expand_dims(img,axis=0)
        if i==0:
            final_predictions=model1.predict(img)
        elif i==1:
            final_predictions=model2.predict(img)
        elif i==2:
            final_predictions=model3.predict(img)
        elif i==3:
            final_predictions=model4.predict(img)
        elif i==4:
            final_predictions=model5.predict(img)
        predictions[idx,...]=final_predictions[0,...]
    loss.append(loss_fn(y_true,predictions))
print(np.mean(loss))        

100%|██████████| 5/5 [03:15<00:00, 39.06s/it]

0.16997722983360292


## with tta

In [6]:
def get_transformed(image_name):
    img=load_img('../input/plant-pathology-2020-fgvc7/'+image_name,target_size=(380,380))
    img=img_to_array(img)/255.0
    transform=a.Compose([a.HorizontalFlip(p=0.5),a.VerticalFlip(p=0.5)])
    transformed=transform(image=img)
    return transformed['image']

In [11]:
rounds=3.0

In [12]:
loss_fn=tf.keras.losses.CategoricalCrossentropy()
loss=[]
for i in tqdm(range(5)):
    temp=df[df['kfolds']==i].drop('kfolds',axis=1).reset_index(drop=True)
    y_true=temp['class'].values
    y_true=tf.one_hot(y_true,depth=4,dtype=tf.float32)
    predictions=np.zeros((y_true.shape[0],4),dtype=float)
    for j in range(int(rounds)):
        for idx,name in enumerate(temp['image_id'].values):
            img=get_transformed(name)
            img=np.expand_dims(img,axis=0)
            if i==0:
                final_predictions=model1.predict(img)
            elif i==1:
                final_predictions=model2.predict(img)
            elif i==2:
                final_predictions=model3.predict(img)
            elif i==3:
                final_predictions=model4.predict(img)
            elif i==4:
                final_predictions=model5.predict(img)
            predictions[idx,...]+=final_predictions[0,...]
    predictions=predictions/rounds
    loss.append(loss_fn(y_true,predictions))
print(np.mean(loss))        

100%|██████████| 5/5 [08:04<00:00, 96.87s/it]

0.16043873727321625


## submission

In [7]:
df_test=pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')

In [8]:
df_test.head()

,image_id
0,Test_0
1,Test_1
2,Test_2
3,Test_3
4,Test_4


In [9]:
sub=pd.read_csv('../input/plant-pathology-2020-fgvc7/sample_submission.csv')
sub.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.25,0.25,0.25,0.25
1,Test_1,0.25,0.25,0.25,0.25
2,Test_2,0.25,0.25,0.25,0.25
3,Test_3,0.25,0.25,0.25,0.25
4,Test_4,0.25,0.25,0.25,0.25


In [10]:
df_test['healthy']=0.0
df_test['multiple_diseases']=0.0
df_test['rust']=0.0
df_test['scab']=0.0

In [14]:
final_predictions=np.zeros((df_test.shape[0],4))
for i in tqdm(range(df_test.shape[0])):
    image_name='images/'+str(df_test.loc[i,'image_id'])+'.jpg'
    for model_idx in range(5):
        model_predictions=np.zeros((1,4))
        if model_idx==0:
            for j in range(3):
                img=get_transformed(image_name)
                img=np.expand_dims(img,axis=0)
                predictions=model1.predict(img)
                model_predictions+=predictions
            final_predictions[i,...]+=model_predictions[0,...]/3.0
        if model_idx==1:
            for j in range(3):
                img=get_transformed(image_name)
                img=np.expand_dims(img,axis=0)
                predictions=model2.predict(img)
                model_predictions+=predictions
            final_predictions[i,...]+=model_predictions[0,...]/3.0
        if model_idx==2:
            for j in range(3):
                img=get_transformed(image_name)
                img=np.expand_dims(img,axis=0)
                predictions=model3.predict(img)
                model_predictions+=predictions
            final_predictions[i,...]+=model_predictions[0,...]/3.0
        if model_idx==3:
            for j in range(3):
                img=get_transformed(image_name)
                img=np.expand_dims(img,axis=0)
                predictions=model4.predict(img)
                model_predictions+=predictions
            final_predictions[i,...]+=model_predictions[0,...]/3.0
        if model_idx==4:
            for j in range(3):
                img=get_transformed(image_name)
                img=np.expand_dims(img,axis=0)
                predictions=model5.predict(img)
                model_predictions+=predictions
            final_predictions[i,...]+=model_predictions[0,...]/3.0

100%|██████████| 1821/1821 [40:28<00:00,  1.33s/it]


In [15]:
final_predictions=final_predictions/5.0

In [16]:
df_test.iloc[:,1:]=final_predictions

In [17]:
#for i in tqdm(range(df_test.shape[0])):
#    image_name=str(df_test.loc[i,'image_id'])
#    img=load_img('../input/plant-pathology-2020-fgvc7/images/'+image_name+'.jpg',target_size=(600,600))
#    img=img_to_array(img)/255.0
#    img=np.expand_dims(img,axis=0)
#    p1=model1.predict(img)
#    p2=model2.predict(img)
#    p3=model3.predict(img)
#    p4=model4.predict(img)
#    p5=model5.predict(img)
#    final_predictions=(p1+p2+p3+p4+p5)/5.0
#    df_test.loc[i,'healthy']=final_predictions[0][0]
#    df_test.loc[i,'multiple_diseases']=final_predictions[0][1]
#    df_test.loc[i,'rust']=final_predictions[0][2]
#    df_test.loc[i,'scab']=final_predictions[0][3]
#    
#    

In [18]:
df_test.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,0.001196,0.014552,0.983446,0.000806
1,Test_1,0.008267,0.072660,0.912578,0.006494
2,Test_2,0.000396,0.000643,0.000239,0.998721
3,Test_3,0.997916,0.000206,0.001392,0.000486
4,Test_4,0.000158,0.008310,0.991308,0.000224


In [19]:
df_test.to_csv('submission.csv',index=False)